In [37]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_dir = 'D://project cps/DATA/known faces'

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(28, 28),
    batch_size=32,
    color_mode='grayscale',
    class_mode='categorical',  # Use 'categorical' for multiple classes
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(28, 28),
    batch_size=32,
    color_mode='grayscale',
    class_mode='categorical',
    subset='validation'
)

Found 6498 images belonging to 10 classes.
Found 1619 images belonging to 10 classes.


In [38]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')  # 9 output classes
])

In [39]:
model.compile(optimizer='adam',
        loss='categorical_crossentropy',  # Categorical loss for multi-class classification
        metrics=['accuracy'])

model.fit(train_generator,
        validation_data=validation_generator,
        epochs=20)  # Adjust epochs as needed

Epoch 1/20
204/204 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.4491 - loss: 1.5155 - val_accuracy: 0.8184 - val_loss: 0.5971
Epoch 2/20
204/204 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9698 - loss: 0.1266 - val_accuracy: 0.9203 - val_loss: 0.3624
Epoch 3/20
204/204 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9843 - loss: 0.0563 - val_accuracy: 0.9413 - val_loss: 0.3881
Epoch 4/20
204/204 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9859 - loss: 0.0542 - val_accuracy: 0.9370 - val_loss: 0.3549
Epoch 5/20
204/204 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9918 - loss: 0.0325 - val_accuracy: 0.9450 - val_loss: 0.3664
Epoch 6/20
204/204 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9920 - loss: 0.0273 - val_accuracy: 0.9475 - val_loss: 0.2828
Epoch 7/20
204/204 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9938 - loss: 0.0180 - val_accuracy: 0.9506 - val_loss: 0.2753
Epoch 8/20
204/204 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9958 - loss: 0.0167 - val_accu

In [40]:
model.save('face_recognition.h5')

# Optionally evaluate the model
test_loss, test_acc = model.evaluate(validation_generator)
print(f"Validation accuracy: {test_acc}")

51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9428 - loss: 0.3262
Validation accuracy: 0.9468808174133301


In [41]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Fine-tuning learning rate reduction callback
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Data augmentation with ImageDataGenerator
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,      # Randomly rotate images
    width_shift_range=0.2,  # Randomly shift images horizontally
    height_shift_range=0.2, # Randomly shift images vertically
    shear_range=0.2,        # Randomly shear images
    zoom_range=0.2,         # Randomly zoom into images
    horizontal_flip=True,   # Randomly flip images horizontally
)

train_generator = datagen.flow_from_directory(
    'D://project cps/DATA/known faces',
    target_size=(28, 28),
    batch_size=32,
    color_mode='grayscale',
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    'D://project cps/DATA/known faces',
    target_size=(28, 28),
    batch_size=32,
    color_mode='grayscale',
    class_mode='categorical',
    subset='validation'
)

# Updated model with Dropout and Batch Normalization
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.25),
    
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.25),
    
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.25),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            loss='categorical_crossentropy',
            metrics=['accuracy'])

# Train the model with callbacks
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=30,  # Increase epochs for better fine-tuning
    callbacks=[reduce_lr, early_stopping]
)


Found 6498 images belonging to 10 classes.
Found 1619 images belonging to 10 classes.
Epoch 1/30
204/204 ━━━━━━━━━━━━━━━━━━━━ 7s 23ms/step - accuracy: 0.3466 - loss: 2.2086 - val_accuracy: 0.0994 - val_loss: 5.5050 - learning_rate: 0.0010
Epoch 2/30
204/204 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.6691 - loss: 1.1467 - val_accuracy: 0.1618 - val_loss: 5.9922 - learning_rate: 0.0010
Epoch 3/30
204/204 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.7618 - loss: 0.8676 - val_accuracy: 0.5398 - val_loss: 1.7787 - learning_rate: 0.0010
Epoch 4/30
204/204 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.8009 - loss: 0.7399 - val_accuracy: 0.6300 - val_loss: 1.5195 - learning_rate: 0.0010
Epoch 5/30
204/204 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.8380 - loss: 0.6276 - val_accuracy: 0.8067 - val_loss: 0.8898 - learning_rate: 0.0010
Epoch 6/30
204/204 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.8662 - loss: 0.5484 - val_accuracy: 0.7270 - val_loss: 1.0768 - learning_rate: 0.0

In [42]:
model.save('face_recognition.h5')

# Optionally evaluate the model
test_loss, test_acc = model.evaluate(validation_generator)
print(f"Validation accuracy: {test_acc}")

51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.9210 - loss: 0.4565
Validation accuracy: 0.9197035431861877


In [43]:
import cv2
import tensorflow as tf
import numpy as np

# Load the trained face recognition model
face_recognition_model = tf.keras.models.load_model('face_recognition.h5')
# Load the trained smile detection model
smile_detection_model = tf.keras.models.load_model('smile_detection_model.h5')

# Define class names for face recognition
class_names = ['AKA', 'athalah', 'citra', 'fatur', 'husain', 'juna', 'lily', 'mitchel', 'sayyid']

# Confidence threshold for recognizing a known face
confidence_threshold = 90.0

# Load Haar cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initialize webcam
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture image.")
        break

    # Convert frame to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # Extract face region of interest
        face_roi = gray_frame[y:y + h, x:x + w]

        # Resize and preprocess the face for face recognition
        resized_face = cv2.resize(face_roi, (28, 28))
        input_image = resized_face.reshape(1, 28, 28, 1) / 255.0

        # Make a prediction for face recognition
        predictions = face_recognition_model.predict(input_image)
        predicted_class = np.argmax(predictions)
        confidence = np.max(predictions) * 100

        # Determine label based on confidence
        if confidence < confidence_threshold:
            label = "Unknown"
        else:
            label = class_names[predicted_class]

        # Make a prediction for smile detection
        smile_prediction = smile_detection_model.predict(input_image)
        smile_label = 'Smiling' if smile_prediction[0][1] > 0.5 else 'Not Smiling'

        # Draw a rectangle around the face and display labels
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        cv2.putText(frame, f"{label} ({confidence:.2f}%)", (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        cv2.putText(frame, smile_label, (x, y + h + 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Display the frame with detections
    cv2.imshow('Face and Smile Recognition', frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━

In [26]:
import cv2
import tensorflow as tf
import numpy as np

# Load the trained model
model = tf.keras.models.load_model('face_recognition.h5')

# Define class names
class_names = ['AKA', 'athalah', 'CIT', 'fatur', 'husain','juna', 'lily', 'mitchel', 'sayyid']

# Confidence threshold
confidence_threshold = 85.0  # Increase threshold to reduce misclassification

# Load Haar cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initialize webcam
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture image.")
        break

    # Convert frame to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # Extract face region of interest
        face_roi = gray_frame[y:y + h, x:x + w]

        # Resize and preprocess the face for the model
        resized_face = cv2.resize(face_roi, (28, 28))
        input_image = resized_face.reshape(1, 28, 28, 1) / 255.0

        # Make a prediction
        predictions = model.predict(input_image)
        predicted_class = np.argmax(predictions)
        confidence = np.max(predictions) * 100

        # Determine label based on confidence
        if confidence < confidence_threshold:
            label = "Unknown"
        else:
            label = class_names[predicted_class]

        # Draw a rectangle around the face and display label
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        cv2.putText(frame, f"{label} ({confidence:.2f}%)", (x, y - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Display the frame with detections
    cv2.imshow('Face Recognition', frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━